In [1]:
using MacroEnergy
using Gurobi

system = MacroEnergy.load_system(@__DIR__)

┌ Info: Loading system from /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/cement_retrofit_locations_test/system_data.json
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system.jl:18
┌ Info: Loading system data
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:18
┌ Info: Done loading system data. It took 0.43 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:26
┌ Info: Generating system
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/generate_system.jl:18
┌ Warning: TotalHoursModeled not found in time_data.json - Using 8760 as default value for TotalHoursModeled
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:79
┌ Warning: Using default weights = 1 as no period map provided and each period maps to itself
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:194
┌ W

MacroEnergy.System("/Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/cement_retrofit_locations_test", (ConstraintScaling = true, WriteSubcommodities = true, OverwriteResults = false, OutputDir = "results", OutputLayout = "long", AutoCreateNodes = true, AutoCreateLocations = true), Dict{Symbol, DataType}(:CementFuel => CementFuel, :Uranium => Uranium, :CO2Captured => CO2Captured, :Electricity => Electricity, :Hydrogen => Hydrogen, :CO2 => CO2, :Coal => Coal, :LiquidFuels => LiquidFuels, :Cement => Cement, :NaturalGas => NaturalGas…), Dict{Symbol, MacroEnergy.TimeData}(:Cement => MacroEnergy.TimeData{Cement}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0), Dict(1 => 1)), :NaturalGas => MacroEnergy.TimeData{NaturalGas}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0), Dict(1 => 1)), :CO2 => MacroEnergy.TimeData{CO2}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0), Dict(1 => 1)), :Electricity => MacroEnergy.TimeData{Electricit

In [2]:
system.assets[1].elec_edge.location

"MA"

In [3]:
system.assets[1].fuel_edge.location

missing

In [4]:
system.assets[1].cement_edge.location

missing

In [5]:
system.assets[1].co2_edge.location

missing

In [9]:
MacroEnergy.location(system.assets[1].cement_edge)

missing